# Test on a different robot
<a href="https://colab.research.google.com/github/jarusgnuj/ioctm358/blob/master/notebooks/time_series_classification/Appendix_2_test_on_robot_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path
import time

import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import model_from_json

# General settings
sns.set_style('whitegrid')

In [ ]:
def load_data(filename):
    ''' Load the data from a file in a GitHub repo '''
    url_root = 'https://raw.githubusercontent.com/jarusgnuj/ai-ml-wksh/master/data/UCR_TSC_archive/SonyAIBORobotSurface2_IoC'
    url = url_root+'/'+filename
    robot_df = pd.read_csv(url, sep='\t', header=None)
    print('Loaded from', url)
    robot_data = robot_df.values
    print('The shape of robot_data is', robot_data.shape)
    return robot_data

In [ ]:
robot_data = load_data('SonyAIBORobotSurface2_TEST.txt')

y = robot_data[:,0].astype(int)
x = robot_data[:,1:]
print('The shape of x is', x.shape)
print('The shape of y is', y.shape)

print('Number of samples of class 0', (y == 0).sum())
print('Number of samples of class 1', (y == 1).sum())
n0 = (y == 0).sum()
n1 = (y == 1).sum()
print('Baseline accuracy to beat:', n1/(n0+n1))

In [ ]:
x_reshape = np.zeros((x.shape[0], 70)) 
x_reshape[:x.shape[0],:x.shape[1]] = x
x_test = x_reshape
y_test = y

In [ ]:
samples = [1, 2, 3, 5, 7]
for i in samples:
    plt.plot(x_test[i], label='class'+str(y_test[i]))
plt.legend(loc='upper right', frameon=False)
plt.ylim([-3.5, 3.5])

## Load model

### To make model files available in a Colab session
+ First save a model by running notebook 4_TSC_final_test.ipynb (set the flag so that it saves the model to file)
  + Open the sidebar menu (using the > tab on the left)
  + Select the Files tab, you will see your model files in there - model.json and model.h5
  + Right click on a model file and select download, select Save File and save the file to your machine.
  + Repeat to download the other model file.
+ Return to this notebook. 
+ Upload a model to this Colab session - 
  + open the sidebar menu, 
  + select the Files tab
  + hit UPLOAD
  + select your model files - the .json and the .h5 file
+ Run the cells below

In [ ]:
json_file = open('model.json', 'r')
loaded_json_model = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_json_model)
# load weights into new model
loaded_model.load_weights('model.h5')
print('Model loaded from file')
optimizer = keras.optimizers.Adam()
loaded_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print('Model compiled')

## Run model

In [ ]:
batch_size = 8
result = loaded_model.evaluate(x_test, y_test, batch_size=batch_size)
print('Accuracy on robot 2 data is', result[1])
y_probability = loaded_model.predict_on_batch(x_test)
y_predicted_class = np.round(y_probability).flatten()
print('Some of the test results:')
print('True', y_test[:23])
print('Pred', y_predicted_class[:23])